In [ ]:
# Contrastive Learning + Zero-Day Detection for IoMT IDS (Memperbaiki SPOOFING)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from pathlib import Path
from collections import Counter
import seaborn as sns

In [ ]:
def load_dataset_from_structure(root_path):
    data = []
    for file in root_path.glob('*/*/*.csv'):
        try:
            df = pd.read_csv(file)

            category = file.parents[1].name  # DDoS, DoS, etc.
            attack = file.parent.name        # DDoS ICMP, DoS TCP, etc.
            label_class = 'Benign' if category.upper() == 'BENIGN' else 'Attack'

            df['category'] = category
            df['attack'] = attack
            df['class'] = label_class

            data.append(df)
        except Exception as e:
            print(f"[ERROR] Failed to read file {file}: {e}")
    return pd.concat(data, ignore_index=True)

# Load train
train_root = Path('../../../Data/CICIoMT2024/train')
train_df = load_dataset_from_structure(train_root)

# Load test
test_root = Path('../../../Data/CICIoMT2024/test')
test_df = load_dataset_from_structure(test_root)

# Cek ringkasan
print("Train set:", train_df.shape)
print(train_df[['category', 'attack', 'class']].value_counts())
print("\nTest set:", test_df.shape)
print(test_df[['category', 'attack', 'class']].value_counts())

# Gabungkan train dan test menjadi satu DataFrame
df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
# === 1. Load Combined Dataset ===
# Asumsikan df sudah ada dari tahap sebelumnya
# Cek jika kolom 'class', 'attack' belum tersedia
if 'class' not in df.columns or 'attack' not in df.columns:
    raise ValueError("Dataset harus memiliki kolom 'class' dan 'attack' sebelum menjalankan skrip ini.")

# Normalisasi teks
df['attack'] = df['attack'].astype(str).str.strip().str.upper()
df['class'] = df['class'].astype(str).str.strip().str.upper()

# === Pisahkan kelas BENIGN dan SPOOFING ===
benign_df = df[df['class'] == 'BENIGN'].copy()
spoofing_df = df[df['attack'] == 'SPOOFING'].copy()

# Gabungkan sebagian serangan lain untuk kontras
known_attacks_df = df[(df['class'] == 'ATTACK') & (df['attack'] != 'SPOOFING')].sample(n=30000, random_state=42)

# === Buat label contrastive ===
benign_df['label'] = 0
known_attacks_df['label'] = 1

contrast_df = pd.concat([benign_df, known_attacks_df], ignore_index=True)

In [ ]:
# === Preprocessing ===
X = contrast_df.select_dtypes(include='number')
feature_cols = X.columns  # Simpan fitur yang digunakan

y = contrast_df['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
# === Latih Encoder untuk Embedding ===
inputs = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu')(inputs)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
embedding = Dense(32, activation='relu', name='embedding')(x)
out = Dense(1, activation='sigmoid')(embedding)

encoder_model = Model(inputs=inputs, outputs=out)
encoder_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
encoder_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64, verbose=1)


In [ ]:
# === Ambil embedding dari layer tengah ===
embedding_model = Model(inputs=encoder_model.input, outputs=encoder_model.get_layer('embedding').output)

# === Embedding untuk benign dan spoofing ===
X_spoof = spoofing_df[feature_cols.intersection(spoofing_df.columns)].copy()
X_spoof_scaled = scaler.transform(X_spoof)
embedding_spoof = embedding_model.predict(X_spoof_scaled)

X_benign = benign_df[feature_cols.intersection(benign_df.columns)].copy()
X_benign_scaled = scaler.transform(X_benign)
embedding_benign = embedding_model.predict(X_benign_scaled)

In [ ]:
# === Visualisasi t-SNE ===
from sklearn.manifold import TSNE
X_embed = np.concatenate([embedding_benign[:1000], embedding_spoof[:1000]])
y_embed = ['BENIGN'] * 1000 + ['SPOOFING'] * 1000

X_tsne = TSNE(n_components=2, perplexity=30, random_state=42).fit_transform(X_embed)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=y_embed, palette=['green', 'red'])
plt.title('Visualisasi Embedding: BENIGN vs SPOOFING')
plt.xlabel('TSNE-1')
plt.ylabel('TSNE-2')
plt.tight_layout()
plt.show()